In [ ]:
# %pip install delta-sharing pyspark
%pip install delta-sharing matplotlib

In [ ]:
import delta_sharing as DakkoDatalake
# from pyspark.sql.functions import sum, col
import matplotlib.pyplot as plt

In [ ]:
share_file_path = "./configs/shareconfig/config.share"

In [ ]:
# Create a SharingClient
client = DakkoDatalake.SharingClient(share_file_path)
 
# List all shared tables.
client.list_all_tables()

In [ ]:
shares = client.list_shares()
 
for share in shares:
  schemas = client.list_schemas(share)
  for schema in schemas:
    tables = client.list_tables(schema)
    for table in tables:
      print(f'name = {table.name}, share = {table.share}, schema = {table.schema}')

In [ ]:
table_url = f"{share_file_path}#poc-client.ethereum.raw_transactions"
 
# Use delta sharing client to load data
pandas_df = DakkoDatalake.load_as_pandas(table_url)
 
pandas_df.head(10)

In [ ]:
# Function to convert hexadecimal strings to decimal integers
def hex_to_decimal(hex_string):
    return int(hex_string, 16)

# Apply the conversion to the Price column
pandas_df['gas_decimal'] = pandas_df['gas'].apply(hex_to_decimal)
pandas_df['gasPrice_decimal'] = pandas_df['gasPrice'].apply(hex_to_decimal)
pandas_df['value_decimal'] = pandas_df['gasPrice'].apply(hex_to_decimal)

# Group by item and calculate mean and sum of price and quantity
grouped_df = pandas_df.groupby('to').agg(mean_price=('gas_decimal', 'mean'), sum_quantity=('gasPrice_decimal', 'sum'))

# Print the grouped DataFrame
print(grouped_df)


In [ ]:

# Create a bar chart of the mean price for each item
plt.bar(grouped_df.index, grouped_df['mean_price'])
plt.xlabel('Item')
plt.ylabel('Mean Price')
plt.title('Mean Price of Items')
plt.show()

In [ ]:
# # Create a pie chart of the sum quantity for each item
plt.pie(pandas_df['gas_decimal'], labels=pandas_df.index, autopct='%1.1f%%')
plt.title('Sum Quantity of Items')
plt.show()